In [4]:
# from pycaret.time_series import *

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import category_encoders as ce

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
import plotly.express as px
%matplotlib inline
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치
# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
# from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor 
from sklearn.ensemble import StackingClassifier, StackingRegressor
# from sklearn.base import ClassifierMixin

# CatBoost
from catboost import CatBoostRegressor

# PyTorch
# import torch
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
# from torch.nn import Parameter
# from torch import Tensor
# from torch.utils.data import DataLoader

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# from bayes_opt import BayesianOptimization
# from num2words import num2words
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

import folium
from geopy.geocoders import Nominatim
import time
import geopandas as gpd
import fiona
def get_coordinates(address):
    geolocator = Nominatim(user_agent = "South Korea")
    location = geolocator.geocode(address)

    if location:
        return location.latitude, location.longitude
    else:
        return 0
path = '../data/daegu/'
external = path+'external_open/대구 빅데이터 마트 데이터/'

In [5]:
# 대구광역시 행정동
daegu_geo1 = gpd.read_file(external+'0. Base/대구광역시_100.gpkg',encoding='cp949')
daegu_geo2 = gpd.read_file(external+'0. Base/대구광역시_300.gpkg',encoding='cp949')
daegu_geo3 = gpd.read_file(external+'0. Base/대구광역시_500.gpkg',encoding='cp949')

daegu_geo = pd.concat([daegu_geo1,daegu_geo2,daegu_geo3],axis=0)
daegu_geo = daegu_geo.to_crs(epsg='4326') # crs 정보 위경도 좌표계로 변경
daegu_geo.set_index('id',inplace=True)
daegu_geo.sort_index(inplace=True)
daegu_geo.reset_index(inplace=True)
daegu_geo.head(2)

,id,left,top,right,bottom,geometry
0,69,1.077033e+06,1.746587e+06,1.077533e+06,1.746087e+06,"MULTIPOLYGON (((128.35675 35.70806, 128.35710 ..."
1,70,1.077033e+06,1.746087e+06,1.077533e+06,1.745587e+06,"MULTIPOLYGON (((128.35180 35.70402, 128.35230 ..."


In [6]:
daegu_geo['centroid'] = daegu_geo['geometry'].centroid
daegu_geo['area'] = daegu_geo['geometry'].area

In [7]:
daegu_geo.head(2)

,id,left,top,right,bottom,geometry,centroid,area
0,69,1.077033e+06,1.746587e+06,1.077533e+06,1.746087e+06,"MULTIPOLYGON (((128.35675 35.70806, 128.35710 ...",POINT (128.35695 35.70808),4.027460e-08
1,70,1.077033e+06,1.746087e+06,1.077533e+06,1.745587e+06,"MULTIPOLYGON (((128.35180 35.70402, 128.35230 ...",POINT (128.35511 35.70523),1.492450e-05


In [8]:
daegu_geo['centroid'][0].x, daegu_geo['centroid'][0].y

(128.35695178788671, 35.708079581069356)

In [9]:
import requests

#### 횡단보도 동별로 개수 구하기
def get_sigudong_from_centroid(point):
    latlong = [point.x,point.y]
    
    # NCP 콘솔에서 복사한 클라이언트ID와 클라이언트Secret 값
    client_id = "y9ig6cz3sg"
    client_secret = "1YHOy21rqo1f1gmJkxkUGD8jTDmIZ7Z81IdtLfcY"

    # 좌표 (경도, 위도)
    coords = f"{latlong[0]},{latlong[1]}"
    output = "json"
    orders = 'addr'
    endpoint = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
    url = f"{endpoint}?coords={coords}&output={output}&orders={orders}"

    # 헤더
    headers = {
        "X-NCP-APIGW-API-KEY-ID": client_id,
        "X-NCP-APIGW-API-KEY": client_secret,
    }

    # 요청
    res = requests.get(url, headers=headers)
    j = res.json()
    
    if j['status']['name'] == 'ok':
        si = j['results'][0]['region']['area1']['name']
        gu = j['results'][0]['region']['area2']['name']
        dong = j['results'][0]['region']['area3']['name']
        
    else:
        si = ''
        gu = ''
        dong = ''
    
    return [si,gu,dong]   

In [10]:
result=daegu_geo['centroid'].apply(get_sigudong_from_centroid)

KeyboardInterrupt: 

In [ ]:
daegu_geo.head(3)